# Import Libraries and Construct Data

In [1]:
from __future__ import division
import pickle
import random
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

In [2]:
raw_data = dict()
raw_data['train_data'] = np.load("training_arr.npy")
raw_data['test_data_pos'] = np.load("biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load("biased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

# Define & Load Model

In [15]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [16]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix="./yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

In [17]:
cml_model.load("cml-yahoo")

INFO:tensorflow:Restoring parameters from bpr-yahoo


# Generate Raw Results

In [18]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = "cml-yahoo-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = "cml-yahoo-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 15400/15400 [05:51<00:00, 43.84it/s]


{'AUC': [0.4620338983050848,
  0.508639455782313,
  0.48566552901023896,
  0.4765166666666666,
  0.4705536912751679,
  0.4943097643097643,
  0.5184364261168385,
  0.5086254295532646,
  0.5179166666666667,
  0.4923063973063973,
  0.4735738255033557,
  0.4655536912751677,
  0.4822297297297298,
  0.5056711409395972,
  0.49600340136054416,
  0.49072881355932196,
  0.5017333333333334,
  0.46940878378378376,
  0.5055518394648828,
  0.47768707482993195,
  0.4979,
  0.49410000000000004,
  0.47662711864406776,
  0.47927852348993283,
  0.4537414965986395,
  0.5114478114478115,
  0.47311224489795917,
  0.518225806451613,
  0.4995317725752509,
  0.4913481228668942,
  0.48451666666666665,
  0.5173578595317726,
  0.532804054054054,
  0.5037037037037038,
  0.49359531772575244,
  0.5056462585034014,
  0.569448160535117,
  0.4432333333333333,
  0.48614999999999997,
  0.4658892617449665,
  0.5627666666666666,
  0.5249833333333334,
  0.5107575757575757,
  0.4629666666666667,
  0.4877999999999999,
  0.482

# Evaluation

In [6]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=30):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [19]:
eq("bpr-yahoo-test-pos-biased_evaluate_partial.pickle", "bpr-yahoo-test-neg-biased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

{'auc': 0.8297434187963931, 'recall': 0.5641305586415502}

In [14]:
eq("cml-yahoo-test-pos-biased_evaluate_partial.pickle", "cml-yahoo-test-neg-biased_evaluate_partial.pickle", "training_arr.npy", gamma=1.5)

{'auc': 0.8450173824825338, 'recall': 0.5989576403918573}